In [1]:
using Random
using Distributions
using FLoops
using Base.Threads

include("data.jl")
include("util.jl")

include("intervals/permutation.jl")
include("intervals/bootstrap.jl")
include("intervals/t.jl")

Threads.nthreads()

112

In [3]:
"""
DATA CONFIG
"""
alpha = 0.05

# data
B  = 100    # num. coverage probabilities per boxplot
S  = 1825   # num. samples per coverage probability
nx = 15      # size of group 1
ny = 8     # size of group 2
px, py = partition(nx, ny)
mc_size = try
    (binomial(nx+ny, nx) > 45_000) ? 10_000 : 0
catch
    0
end

# ALWAYS USE EXACT ENUMERATION FOR NOW
# mc_size = 0

dtype = Float32
seed = 123

"""
POPULATION SETTINGS
"""

Random.seed!(seed)

distrTypeX = Normal{dtype}
X_mu = dtype.(round.(rand(B), digits=3))
X_sigma = dtype.(round.(rand(B), digits=3))
distrX = map(distrTypeX, X_mu, X_sigma)

distrTypeY = Normal{dtype}
Y_mu = dtype.(X_mu .+ ((round.(rand(B), digits=3) * 4) .- 2))
Y_sigma = X_sigma
distrY = map(distrTypeY, Y_mu, Y_sigma)

deltas = @. mean(distrX) - mean(distrY)

@show distrX[1:2]
@show distrY[1:2]
@show deltas[1:2];

distrX[1:2] = Normal{Float32}[Normal{Float32}(μ=0.906f0, σ=0.954f0), Normal{Float32}(μ=0.443f0, σ=0.846f0)]
distrY[1:2] = Normal{Float32}[Normal{Float32}(μ=2.766f0, σ=0.954f0), Normal{Float32}(μ=-1.273f0, σ=0.846f0)]
deltas[1:2] = Float32[-1.86, 1.716]


In [4]:
Random.seed!(123)
xs = [dtype.(rand(distrX[i], nx, S)) for i in 1:B]
ys = [dtype.(rand(distrY[i], ny, S)) for i in 1:B]
@show size(ys)
ys[2]

size(ys) = (100,)


8×1825 Matrix{Float32}:
 -2.28166   -1.1419    -3.18769  …  -1.73181   -0.948724  -1.76034
 -1.23177   -2.2025    -1.23641      0.48103   -1.59983   -1.3861
 -0.781374  -0.600414  -1.74909      0.2314    -2.1528    -3.04988
 -1.7223    -1.57811   -2.2822      -1.63318   -1.35202   -0.336312
 -1.70014   -0.952603  -1.1475      -1.68727   -2.91257   -0.638246
 -2.74712   -0.376529  -1.85082  …  -2.01309   -0.804211  -2.42942
 -3.0038    -2.73733   -1.04156     -0.636508  -2.33025   -0.750896
 -1.97228   -1.66221   -1.3331      -1.02326   -1.57945   -1.06126

In [5]:
# flatten into 3D matrix
X = reshape(hcat(xs...), nx, S, B)
Y = reshape(hcat(ys...), ny, S, B)
Y[:,:,2]

8×1825 Matrix{Float32}:
 -2.28166   -1.1419    -3.18769  …  -1.73181   -0.948724  -1.76034
 -1.23177   -2.2025    -1.23641      0.48103   -1.59983   -1.3861
 -0.781374  -0.600414  -1.74909      0.2314    -2.1528    -3.04988
 -1.7223    -1.57811   -2.2822      -1.63318   -1.35202   -0.336312
 -1.70014   -0.952603  -1.1475      -1.68727   -2.91257   -0.638246
 -2.74712   -0.376529  -1.85082  …  -2.01309   -0.804211  -2.42942
 -3.0038    -2.73733   -1.04156     -0.636508  -2.33025   -0.750896
 -1.97228   -1.66221   -1.3331      -1.02326   -1.57945   -1.06126

In [6]:
function save_ci_results(results, methodId, B, S, pooled=nothing, two_sided=nothing; prefix="", dir="./")
    averages = []
    
    for batchId in 1:B
        batch = results[methodId, batchId, :]
        coverage = sum([j for (j, _) in batch]) / S
        width = sum([j for (_, j) in batch]) / S
        push!(averages, (coverage, width))
    end

    if isnothing(two_sided)
        save(averages, distrX[1:B], distrY[1:B], alpha, prefix=prefix, dir=dir)
    else
        alpha_ = two_sided ? alpha : alpha / 2
        save(averages, distrX[1:B], distrY[1:B], alpha_, pooled, two_sided, prefix=prefix, dir=dir)
    end
end

function save_permutation_results(results, B, S; prefix="", dir="./")
    i = 1
    per_method = []
    for two_sided in [true, false]
        for pooled in [true, false]
            save_ci_results(results, i, B, S, pooled, two_sided, prefix=prefix, dir=dir)
            i += 1
        end
    end
end

save_permutation_results (generic function with 1 method)

In [7]:
B = 100
S = 1825

T = Threads.nthreads()
results = Array{Union{Nothing, Tuple}, 3}(undef, 6, B, S)
fill!(results, nothing)
mc_size = 0

#@time Threads.@threads for (i,j) in collect(Iterators.product(1:B, 1:S)) # 15.52 sec on (B,S) = (5, 1800)
@time @floop ThreadedEx(basesize=div(B*S, T)) for b in 1:B, s in 1:S
    @inbounds x = X[:,s,b]
    @inbounds y = Y[:,s,b]

    """ 
    results[1, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), true, alpha, twoSided, twoSided)    
    results[2, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), false, alpha, twoSided, twoSided)
    results[3, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), true, alpha/2, greater, smaller, 0.0005)
    results[4, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), false, alpha/2, greater, smaller, 0.0005)
    """
    
    results[5, b, s] = bootstrap(x, y, deltas[b], alpha, nsamples=10_000)
    """
    results[6, b, s] = tconf(x, y, deltas[b], alpha, false)
    """
end

dir = "../results/" * string(nx) * "_" * string(ny) * "/1/"
save_ci_results(results, 5, B, S, true; prefix="bs", dir=dir)
#save_permutation_results(results, B, S; dir=dir)

2534.511027 seconds (3.84 G allocations: 408.328 GiB, 93.69% gc time, 0.20% compilation time)
dir * filename = "../results/15_8/1/bs_0.05.csv"


"../results/15_8/1/bs_0.05.csv"

In [76]:
averages = []
for row in eachrow(results[5,:,:])
    coverage = sum([j for (j, _) in row]) / S
    width = sum([j for (_, j) in row]) / S
    push!(averages, (coverage, width))
end
averages

5-element Vector{Any}:
 (0.9041095890410958, 1.7169137288119696)
 (0.8964383561643836, 1.5188572278067674)
 (0.9057534246575343, 1.0491979467480967)
 (0.8997260273972603, 0.21909246020121118)
 (0.9145205479452054, 0.2478563258378473)

---

In [70]:
x = X[:,1,1]
y = Y[:,1,1]
@time permInterval(x, y, deltas[1], (px, py, 0), true, alpha, twoSided, twoSided)

permInterval(x, y, deltas[1], (px, py, 0), true, alpha/2, greater, smaller)

pval(x, y, 2.4218399175319254, (px, py, 0), true, twoSided)

  0.292847 seconds (510.31 k allocations: 55.944 MiB, 93.62% compilation time)


0.05034965f0